# Functional Class and Curvature Analysis
This notebook illustrates the complete analysis example using Here Map Content (HMC) attributes.

### Dependencies
* Catalogs: [rib-2](https://platform.here.com/data/hrn:here:data::olp-here:rib-2)
* Layers: [Topology Geometry](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/topology-geometry), [Road Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/road-attributes), [ADAS Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/adas-attributes)

### Workflow
* Start with a bounding box that contains portions of two tiles
* Extract geometry for the two tiles
* Keep only the segments that are (at least partially) in original bounding box
* Extract Functional Class (FC) attributes for those segments
* Select segments with FC = 3, 4, and 5
* Extract curvature for the survived segments
* Analyze average curvature
* Drop segments below the average curvature

## Set API key (here.access.key.secret)

In [ ]:
import os
os.environ["LS_API_KEY"] = "YOUR HERE API KEY"

## Get partition ID's from a given bounding box

In [ ]:
from here.geotiles.heretile import in_bounding_box
from shapely.geometry import Point

zoom_level = 12
west = 13.41
east = 13.49
south = 52.50
north = 52.54
center = Point(west, south)
tileIds = list(in_bounding_box(west=west, south=south, east=east, north=north, level=zoom_level))

# partition ID's: string representation of tile IDs - this will be used for extracting attributes later
partition_ids = [str(tile) for tile in tileIds]
print("partition ids:", partition_ids)

## Visualize the bounding box and tile boundaries
- bounding box: Red
- tile boundaries: Gray

In [ ]:
from here.inspector import inspect
from here.inspector.styles import Color
from shapely.geometry import Polygon

# construct a bounding box geojson
bb_polygon = Polygon([(west, south), (west, north), (east, north), (east, south), (west, south)])

# create inspect object with tile boundaries and the bounding box
inspect(layers={"bounding_box": [bb_polygon]},
        tiles=tileIds,
        center=center,
        zoom=11,
        layers_style={"bounding_box": Color.RED},
        tiles_style=Color.GRAY)

## Extract topology geometry and clip by bounding box

In [ ]:
from here.platform import Platform
from here.content.hmc2.hmc import HMC
from here.geopandas_adapter import GeoPandasAdapter
import pandas as pd
import geopandas as gpd

# create HMC object
hmc = HMC(Platform(), adapter=GeoPandasAdapter())

In [ ]:
# get segments dataframe from given tiles
segment_df = hmc.topology_geometry.get(tileIds, "segment")
segment_df

### Clip segments by the bounding box
Since the geometry objects for the segment dataframe is LineString, the intersection method will do the clipping. See the documentation of shapely: https://shapely.readthedocs.io/en/stable/manual.html

In [ ]:
clipped_segments = segment_df.intersection(bb_polygon)
segment_in_bb = segment_df[clipped_segments.apply(lambda x: x.length != 0)]
segment_in_bb

In [ ]:
# show the clipped segments and original tile boundaries
inspect(features=segment_in_bb["geometry"], tiles=tileIds, tiles_style=Color.GRAY)

## Get functional class attribute with segment reference

In [ ]:
# get functional_class attributes
fc_df_seg = hmc.road_attributes.get_referencing(tileIds, "functional_class", "segment")
fc_df_seg

## Select FC = 3, 4, or 5

In [ ]:
# select FC = 3, 4, 5
fc345_df = fc_df_seg[
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_3") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_4") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_5")
]
fc345_df

In [ ]:
# change indexes into actual columns
fc345_df = fc345_df.reset_index()
# rename columns for easier merging later on
fc345_df.rename(columns={'ref_partition': 'partition_id'}, inplace=True)
fc345_df.rename(columns={'ref_identifier': 'segment_id'}, inplace=True)
fc345_df

In [ ]:
# join two dataframe by partitionId, segmentId
segment_fc345_df = segment_in_bb.merge(fc345_df,
                                       how="inner",
                                       on=["partition_id", "segment_id"])
segment_fc345_df

## Show the segments with FC = 3, 4, or 5

In [ ]:
inspect(segment_fc345_df["geometry"])

## Show different colors for each FC

In [ ]:
fcs = {}
for fc, gdf in segment_fc345_df.groupby("attribute.functional_class"):
    fcs[fc] = gdf["geometry"]
inspect(layers=fcs)

## Extract curvatures

In [ ]:
# get segment curvatures (curvatures at the nodes are not used in this notebook example)
segment_curvature_df = hmc.adas_attributes.get(partition_ids, "curvature_heading")
segment_curvature_df

## Select segments with high curvatures

In [ ]:
# create a column called abs_curvature to calculate average curvature
segment_curvature_df["abs_curvature"] = segment_curvature_df.apply(lambda x: abs(x["curvature"]), axis=1)
average_curvature = segment_curvature_df["abs_curvature"].mean()

# select segments with curvature higher than average
highcurv_segments = segment_curvature_df[segment_curvature_df["abs_curvature"] > average_curvature]

# extract segment_id from segment anchors
highcurv_segments = highcurv_segments.copy()
highcurv_segments["segment_id"] = highcurv_segments.apply(
    lambda x: x["segment_anchors"][0].oriented_segment_ref[0][0].segment_id,
    axis=1
)

highcurv_segments

## Join two dataframes
Inner join between the dataframe with FC=3,4,5 and the dataframe with high curvatures

In [ ]:
segment_fc345_highcurv_df = segment_fc345_df.merge(highcurv_segments,
                                                   how="inner",
                                                   on=["partition_id", "segment_id"])
segment_fc345_highcurv_df

In [ ]:
# drop abs_curvature which was used to calculate average curvature
merged_df = segment_fc345_highcurv_df.drop("abs_curvature", axis=1)
merged_df.columns

## Show the survived segments

In [ ]:
inspect(merged_df["geometry"])

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>